In [1]:
from codebase.classes_ibis_lvm import ParticlesLVM
from codebase.classes_data import Data
from codebase.ibis import model_phonebook, essl
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis_lvm import run_ibis_lvm

from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
from codebase.plot import plot_density, get_post_df, plot_correlations
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## 2 factor Sim EZ

In [2]:
existing_directory = None
task_handle = 'ez_lvm'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)

# log_dir = './log/20201230_133053_ez_lvm/'



Creating new directory: ./log/20210101_205543_ez_lvm/


## Create Data

In [4]:

# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 1,  
    size = 30,
    random_seed = 0
    )
    
exp_data.generate()
save_obj(exp_data, 'data', log_dir)


## Run IBIS

In [6]:
model_num = 7

ibis = run_ibis_lvm(
    exp_data,
    model_num,
    100,
    50,
    gen_model,
    log_dir
    )


100%|██████████| 30/30 [02:28<00:00,  4.95s/it]




Marginal Likelihood 0.78640


In [10]:
ibis['jitter_corrs']['beta']

array([-0.05037161,  0.08990666,  0.15160443,  0.        ,  0.18398819,
        0.        ,  0.        ,  0.32246262,  0.        ,  0.30951299,
        0.3127969 ,  0.        ,  0.28915436,  0.        ,  0.        ,
        0.27565835,  0.        ,  0.        ,  0.39031765,  0.        ,
        0.45997703,  0.        ,  0.        ,  0.39472646,  0.        ,
        0.        ,  0.        ,  0.45083163,  0.        ,  0.        ])

## Plot Both results

In [11]:
# have to resample particles to get rid of weights
particles = ibis['particles']
particles.resample_particles()

In [12]:
np.mean(np.squeeze(particles.particles['beta']),0)

array([ 0.1471664 , -0.03145514,  0.15082876,  0.09419411,  0.19056473,
        0.22028841])

In [13]:
particles.particles[param]

NameError: name 'param' is not defined

In [15]:
param = 'beta'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(df2)  &\
plot_correlations(ibis['jitter_corrs'][param], height = 150)

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.VConcatChart(...)

In [ ]:
param = 'alpha'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 